In [21]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/flood_data.csv")
df["Start_Date"] = pd.to_datetime(df["Start_Date"])
df["End_Date"] = pd.to_datetime(df["End_Date"])

date = "End_Date"
df.loc[
    (df[date] >= pd.Timestamp("2024-09-09")) & (df[date] <= pd.Timestamp("2024-09-11"))
]


,Admin3,Admin2,Admin1,Observed_Area_SqKM,Flooded_Area_SqKM,Population_Exposed,Flooded Crop Area,Start_Date,End_Date,Latitude,Longitude
256,Waw,Bago,Bago (East),1012.86,495.08,62814,428.90,2024-07-01,2024-09-09,17.336667,96.479722
257,Nyaunglebin,Bago,Bago (East),802.35,394.09,63925,353.76,2024-07-01,2024-09-09,17.336667,96.479722
258,Daik-U,Bago,Bago (East),1351.13,392.03,49057,353.56,2024-07-01,2024-09-09,17.336667,96.479722
259,Bago,Bago,Bago (East),3121.00,289.26,57139,206.01,2024-07-01,2024-09-09,17.336667,96.479722
260,Thanatpin,Bago,Bago (East),875.40,273.22,34243,241.97,2024-07-01,2024-09-09,17.336667,96.479722
...,...,...,...,...,...,...,...,...,...,...,...
369629,Salingyi,Yinmarbin,Sagaing,56.08,0.07,6,0.07,2024-09-06,2024-09-09,22.077789,94.900139
369630,Kani,Yinmarbin,Sagaing,3826.14,6.52,461,2.58,2024-09-06,2024-09-11,22.077789,94.900139
369631,Pale,Yinmarbin,Sagaing,1844.61,3.65,366,2.64,2024-09-06,2024-09-11,22.077789,94.900139
369632,Yinmarbin,Yinmarbin,Sagaing,901.14,1.44,205,0.70,2024-09-06,2024-09-11,22.077789,94.900139


In [ ]:
def interpolate_df(df: pd.DataFrame, admin_level: int) -> pd.DataFrame:
    """
    Interpolate the given data to fill in missing values.

    Params
    ------
    - df (pd.DataFrame): The data to interpolate.
    - admin_level (int): The admin level to interpolate the data for. Can only be 1, 2, or 3.

    Returns
    -------
    - pd.DataFrame: The interpolated data.
    """
    assert admin_level in [1, 2, 3], "Admin level is not 1, 2, or 3."

    # Group by the admin level and end date and sum the flooded area to pivot the data
    pivot_df = df.groupby(["Admin2", "End_Date", "Latitude", "Longitude"])["Flooded_Area_SqKM"].sum().reset_index()
    pivot_df["End_Date"] = pd.to_datetime(pivot_df["End_Date"])
    pivot_df = pivot_df.pivot(index=["Admin2", "Latitude", "Longitude"], columns="End_Date", values="Flooded_Area_SqKM")

    full_date_range = pd.date_range(start=pivot_df.columns.min(), end=pivot_df.columns.max(), freq="D")
    pivot_df = pivot_df.reindex(columns=full_date_range, fill_value=np.nan)

    interpolate_df = pivot_df.interpolate(method="linear", axis=1)
    interpolate_df = interpolate_df.reset_index()
    interpolate_df = interpolate_df.round(2)
    interpolate_df.fillna(0, inplace=True)

    return interpolate_df

In [40]:
interpolate_df(df, 2)


KeyError: 'Latitude'